In [9]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

import matplotlib.pyplot as plt
import scipy.io
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

from sklearn.manifold import TSNE
import seaborn as sns

import pandas as pd
import xlsxwriter
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

from sklearn.decomposition import PCA
from sklearn.decomposition import SparsePCA

tf.get_logger().setLevel('ERROR')

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
#read the data into a pandas dataframe
df = pd.read_csv("bert_data_withoutClassWord.csv")
print(df.shape)
df.head(5)

(7115, 3)


,Unnamed: 0,text,class
0,0,this article is about the herbivorous mammals....,Antelope
1,1,"one new world species, the pronghorn of north ...",Antelope
2,2,"the english word ""animal"" first appeared in 14...",Antelope
3,3,"the word talopus and calopus, from latin, came...",Antelope
4,4,animal are not a cladistic or taxonomically de...,Antelope


In [3]:
#Add the new column which gives a unique number to each of these labels 

j = 0
for i in df['class'].unique():
    df.loc[df['class'] == i, ['class_num']] = j
    j += 1

#checking the results 
df.head(50000)

,Unnamed: 0,text,class,class_num
0,0,this article is about the herbivorous mammals....,Antelope,0.0
1,1,"one new world species, the pronghorn of north ...",Antelope,0.0
2,2,"the english word ""animal"" first appeared in 14...",Antelope,0.0
3,3,"the word talopus and calopus, from latin, came...",Antelope,0.0
4,4,animal are not a cladistic or taxonomically de...,Antelope,0.0
...,...,...,...,...
7110,7110,technology to use sponges as mouth protection ...,dolphin,49.0
7111,7111,"pesticides, heavy metals, plastics, and other ...",dolphin,49.0
7112,7112,"hundreds of orcas, animals and other members o...",dolphin,49.0
7113,7113,captured orcas and animals are confined to tan...,dolphin,49.0


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.text, 
    df.class_num, 
    test_size=0.1, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df.class_num
)

In [5]:
embbeding_size = 4000
clustering_count = 3
numof_embb = 2
att_size = embbeding_size * clustering_count

In [19]:
acc = 0

while 0.519 > acc:
    
    vectorizer = TfidfVectorizer(max_features = embbeding_size).fit(X_train)
    
    len(vectorizer.get_feature_names_out())
    
    X_train_vectorized = vectorizer.transform(X_train)
    X_train_vectorized = X_train_vectorized.toarray()
    
    #using pca
    S_pca = SparsePCA(n_components=1500, random_state=0)
    X_train_PCA = S_pca.fit_transform(X_train_vectorized)    
    
    clf = RandomForestClassifier()
    clf.fit(X_train_PCA, y_train)
    
    #transforming X_test
    X_test_vectorized = vectorizer.transform(X_test)
    X_test_vectorized = X_test_vectorized.toarray()
    X_test_PCA = S_pca.transform(X_test_vectorized)
    
    predictions = clf.predict(X_test_PCA)
    acc = accuracy_score(y_test, predictions)
    print("ACC:", accuracy_score(y_test, predictions))

ACC: 0.43679775280898875
ACC: 0.4438202247191011
ACC: 0.425561797752809
ACC: 0.44803370786516855


KeyboardInterrupt: 

In [26]:
# vectorizer.transform() is our emb creator

all_atts = vectorizer.transform(df.text).toarray()
print(type(all_atts))
print(all_atts[0])
all_labels = df['class']

<class 'numpy.ndarray'>
[0. 0. 0. ... 0. 0. 0.]


In [27]:
all_atts[0].shape

(4000,)

In [28]:
#print(all_atts)
data = pd.DataFrame(all_atts)

data['labels'] = all_labels
data

,0,1,2,3,4,5,6,7,8,9,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,labels
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Antelope
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Antelope
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Antelope
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Antelope
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Antelope
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dolphin
7111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dolphin
7112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dolphin
7113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dolphin


In [29]:
def att_generator(embbedings, numof_embb=numof_embb, num_clusters=clustering_count):

    #clustering embbedings
    km = KMeans(n_clusters=num_clusters)
    clustering_predictions = km.fit_predict(embbedings)
    
    #convert to dataframe for ease of use
    df = pd.DataFrame(embbedings)
    
    #include the cluster labels
    df["clusters"] = clustering_predictions
    
    #creating the attribute array
    the_attribute = np.array([])
    
    for i in np.unique(clustering_predictions):
        
        cluster_embbedings = df[df["clusters"] == i]
        cluster_embbedings = cluster_embbedings.drop(["clusters"], axis=1) 
        cluster_embbedings_nparray = cluster_embbedings.to_numpy()
        
        cluster_mean = np.mean(cluster_embbedings_nparray, axis=0)
        emb_distances = np.array([])
        
        #calculating each embbeding distance from its cluster mean and selecting the closest one to the mean
        for i in range(cluster_embbedings_nparray.shape[0]):
            
            dist = np.linalg.norm(cluster_mean - cluster_embbedings_nparray[i])
            emb_distances = np.append(emb_distances, dist)
            
            
        
        emb_distances = np.reshape(emb_distances, (-1, 1))
        #print(cluster_embbedings_nparray.shape, emb_distances.shape)
        temp = np.concatenate((cluster_embbedings_nparray, emb_distances), axis=1)
        #print(temp.shape)
        temp = temp[temp[:, -1].argsort()]
        #print(temp[:, -1])
        sorted_embbedings = np.delete(temp, -1, 1)
        #print(sorted_embbedings.shape)
        
        the_attribute = np.concatenate((the_attribute, sorted_embbedings[0]), axis=0)
        print(the_attribute.shape)

    
    return the_attribute

In [34]:
unique_classes = data["labels"].unique()
attribute_array = np.zeros([1, att_size])

for i in unique_classes:
    
    class_embbedings = data.loc[data['labels'] == i]
    class_embbedings = class_embbedings.drop(['labels'], axis=1)    

    myatt = att_generator(class_embbedings)
    myatt = myatt.reshape(1, att_size)
    
    attribute_array = np.append(attribute_array, myatt, axis=0)

attribute_array = np.delete(attribute_array, 0, 0)
attribute_array = attribute_array.transpose()

C:\Users\MOHSEN\anaconda3\envs\piptext\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


(4000,)
(8000,)
(12000,)
(4000,)
(8000,)
(12000,)


C:\Users\MOHSEN\anaconda3\envs\piptext\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\MOHSEN\anaconda3\envs\piptext\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


(4000,)
(8000,)
(12000,)


C:\Users\MOHSEN\anaconda3\envs\piptext\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\MOHSEN\anaconda3\envs\piptext\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


...

C:\Users\MOHSEN\anaconda3\envs\piptext\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


(4000,)
(8000,)
(12000,)


C:\Users\MOHSEN\anaconda3\envs\piptext\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


(4000,)
(8000,)
(12000,)
(4000,)
(8000,)
(12000,)


C:\Users\MOHSEN\anaconda3\envs\piptext\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [35]:
the_mat_file = scipy.io.loadmat('att_splits_AWA2.mat')
the_mat_file["att"] = attribute_array

print(the_mat_file["att"].shape)

(12000, 50)


In [36]:
scipy.io.savemat('tf-idf_4000_embClust(3,1).mat', the_mat_file)

In [214]:
#data = data.rename(columns={"0": "embbedings"})
#data
data.loc[data['labels'] == 'Antelope']

,embbedings,labels
0,"(0, 3815)\t0.2116568711329839\n (0, 3798)\t...",Antelope
1,"(0, 3972)\t0.190382191693978\n (0, 3725)\t0...",Antelope
2,"(0, 3966)\t0.36810011017665584\n (0, 3925)\...",Antelope
3,"(0, 3966)\t0.189929209485221\n (0, 3878)\t0...",Antelope
4,"(0, 3954)\t0.1671060571283198\n (0, 3758)\t...",Antelope
...,...,...
185,"(0, 3796)\t0.15071770983014868\n (0, 3758)\...",Antelope
186,"(0, 3942)\t0.13666177470169633\n (0, 3595)\...",Antelope
187,"(0, 3883)\t0.16809701621305914\n (0, 3595)\...",Antelope
188,"(0, 3798)\t0.26935198636538843\n (0, 3747)\...",Antelope


# obtain att

In [163]:
data = pd.read_csv(r'bert_data_withoutClassWord.csv')

In [164]:
data['class']

0       Antelope
1       Antelope
2       Antelope
3       Antelope
4       Antelope
          ...   
7110     dolphin
7111     dolphin
7112     dolphin
7113     dolphin
7114     dolphin
Name: class, Length: 7115, dtype: object

In [134]:
the_mat_file = scipy.io.loadmat('att_splits_AWA2.mat')
the_mat_file['att'].shape

(85, 50)